## Presidential results (D%, R%, two-party %)

Difficulty: Easy

Sources: MIT Election Data & Science Lab (county & state returns), state SOS

MIT Election Lab
Quality issues: late-certified adjustments are rare but check state certification dates. Third-party vote handling—compute two-party share.
Estimated time: hours to 1 day using MIT Election Lab CSVs.

https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi:10.7910/DVN/42MVDX

In [ ]:
#!pip install pandas

In [ ]:
# pip install lxml

In [ ]:
#!pip install pyjanitor

In [ ]:
#!pip install duckdb

In [ ]:
import pandas as pd
df = pd.read_csv("data/POTUS/1976-2020-president.csv")

In [ ]:
print(df)

In [ ]:
import pandas as pd

# === 1. Load the data ===
df = pd.read_csv("data/POTUS/1976-2020-president.csv")

df["state_fips"] = df["state_fips"].astype(str).str.zfill(2)

# Normalize column names just in case (some datasets differ in case)
df.columns = df.columns.str.lower()

# === 2. Keep only relevant columns ===
cols = ["year", "state", "state_po", "state_fips", "party_detailed", "candidatevotes", "totalvotes"]
df = df[cols].rename(
    columns = {"candidatevotes": "candidate_votes", "totalvotes": "total_votes"}
)

# === 3. Group parties into Dem, Rep, or Other ===
df["party_grouped"] = df["party_detailed"].apply(
    lambda x: (
        "dem" if "democrat" in str(x).lower()
        else "rep" if "republican" in str(x).lower()
        else "other"
    )
)

# === 4. Aggregate votes by year, state, and party group ===
pivot_df = (
    df.groupby(["year", "state", "state_po", "state_fips", "party_grouped"])["candidate_votes"]
    .sum()
    .unstack(fill_value=0)
    .reset_index()
)

# === 5. Merge in total votes (max should be same per state-year) ===
total_votes = df.groupby(["year", "state", "state_po", "state_fips"])["total_votes"].max().reset_index()

merged = pivot_df.merge(total_votes, on=["year", "state", "state_po", "state_fips"], how="left")

# === 6. Compute party vote percentages ===
for party in ["dem", "rep", "other"]:
    merged[f"{party.lower()}_pct"] = (merged[party]*100 / merged["total_votes"]).round(2)

# === 7. Compute Dem–Rep difference ===
merged["d_r_diff"] = (merged["dem_pct"] - merged["rep_pct"]).round(2)

# === 8. Optional: sort and inspect ===
merged = merged.sort_values(["year", "state_po"]).reset_index(drop=True)


#print(merged.head())
display(merged)
# === 9. Optional: save output ===
merged.to_csv("data/potus_votes_bystate.csv", index=False)


In [ ]:
nation = (
    merged.groupby("year", as_index=False)
      .agg({
          "dem": "sum",
          "rep": "sum",
          "other": "sum",
          "totalvotes": "sum"
      })
      .assign(
          dem_pct=lambda d: (100 * d.dem / d.totalvotes).round(2),
          rep_pct=lambda d: (100 * d.rep / d.totalvotes).round(2),
          other_pct=lambda d: (100 * d.other / d.totalvotes).round(2),
          d_r_diff=lambda d: (d.dem_pct - d.rep_pct).round(2)
      )
)

print(nation.head())
nation.to_csv("data/potus_votes_national.csv", index=False)

# 2024 POTUS election data

Source: FEC.gov [Excel file](https://view.officeapps.live.com/op/view.aspx?src=https%3A%2F%2Fwww.fec.gov%2Fresources%2Fcms-content%2Fdocuments%2F2024presgeresults.xlsx&wdOrigin=BROWSELINK)

In [ ]:
import pandas as pd
import janitor
df24 = pd.read_excel("data/POTUS/2024presgeresults.xlsx", header=0, nrows=51)
df24 = df24.clean_names()

df24 = df24[["state", "electoral_votes", "electoral_vote_trump_r_", "electoral_vote_harris_d_", "harris", "trump", "total_votes"]].rename(
    columns = {"state": "state_po", "electoral_votes": "ev_total", "electoral_vote_trump_r_": "ev_rep", "electoral_vote_harris_d_": "ev_dem"
               , "harris": "dem", "trump":"rep"}
)

num_cols = [c for c in df24.columns if c != "state_po"]
df24[num_cols] = df24[num_cols].apply(pd.to_numeric, errors="coerce").fillna(0).astype(int)

# calc xx_pct columns
for party in ["dem", "rep"]:
    df24[f"{party.lower()}_pct"] = (df24[party]*100 / df24["total_votes"]).round(2)

# calc diffs
df24["d_r_diff"] = (df24["dem_pct"] - df24["rep_pct"]).round(2)
df24["r_d_diff"] = (df24["rep_pct"] - df24["dem_pct"]).round(2)

# add "year" and move to first column
df24["year"] = 2024
col = "year"
df24 = df24[[col] + [c for c in df24.columns if c != col]]

df24



## Electoral College

### 1976-2020 compiled
https://huggingface.co/datasets/fdaudens/us-presidential-elections-with-electoral-college

### 2024 data from archives.gov
https://www.archives.gov/electoral-college/202

# 2024 Electoral College - web scrape
from https://www.archives.gov/electoral-college/202


In [ ]:
import re
import time
import unicodedata
import requests
import pandas as pd
from bs4 import BeautifulSoup
from io import StringIO

# ===== Candidate → Party map (1976–2024) =====
# Use LAST NAMES (normalized) as keys; mapping happens on normalized header text.
BASE_MAP_BY_YEAR = {
    1976: {'carter':'DEM','ford':'REP','reagan':'REP'},  # WA faithless to Reagan (REP)
    1980: {'carter':'DEM','reagan':'REP','anderson':'OTHER'},
    1984: {'mondale':'DEM','reagan':'REP'},
    1988: {'dukakis':'DEM','bush':'REP','bentsen':'DEM'},  # WV faithless to Bentsen (DEM)
    1992: {'clinton':'DEM','bush':'REP','perot':'OTHER'},
    1996: {'clinton':'DEM','dole':'REP','perot':'OTHER'},
    2000: {'gore':'DEM','bush':'REP'},
    2004: {'kerry':'DEM','bush':'REP','edwards':'DEM'},    # MN faithless to Edwards (DEM)
    2008: {'obama':'DEM','mccain':'REP'},
    2012: {'obama':'DEM','romney':'REP'},
    2016: {
        'clinton':'DEM','trump':'REP',
        'powell':'REP','kasich':'REP','paul':'REP',
        'sanders':'OTHER','faith spotted eagle':'OTHER','spotted eagle':'OTHER','eagle':'OTHER'
    },
    2020: {'biden':'DEM','trump':'REP'},
    2024: {'harris':'DEM','trump':'REP'}
}

# ===== Helpers =====
def flatten_cols(cols):
    out = []
    for c in cols:
        if isinstance(c, tuple):
            c = " ".join([str(x) for x in c if x is not None])
        out.append(str(c))
    return out

def normalize_header(s: str) -> str:
    """
    Normalize NARA header strings so candidate names are detectable:
    - ascii fold (remove accents)
    - lower-case
    - collapse whitespace
    - remove punctuation
    - drop phrases like 'for president', 'president', 'vice president'
    - drop trailing ', of <state>...' fragments
    """
    if s is None:
        return ""

    # ASCII fold and lowercase
    s = unicodedata.normalize("NFKD", s)
    s = "".join(ch for ch in s if not unicodedata.combining(ch)).lower()

    # Replace NBSP etc. with spaces, collapse whitespace
    s = re.sub(r"\s+", " ", s.replace("\xa0", " ")).strip()

    # Remove obvious prefixes
    s = re.sub(r"^(for\s+)?(vice\s+)?president\s+", "", s)

    # Remove ", of <place>..." tail
    s = re.sub(r",?\s+of\s+.+$", "", s)

    # Remove non-letters except spaces (keep hyphens for names like 'spotted eagle')
    s = re.sub(r"[^a-z\s-]", "", s).strip()

    return s

def find_target_table(dfs):
    for df in dfs:
        cols = flatten_cols(df.columns)
        if any(re.search(r"\bstate\b", c, re.I) for c in cols) and \
           any(re.search(r"\belectoral|votes?\b", c, re.I) for c in cols):
            df2 = df.copy()
            df2.columns = cols
            return df2
    return None

def classify_candidate(col_name_raw: str, year: int) -> str:
    """
    Classify a candidate column into DEM/REP/OTHER using normalized header text.
    Works even when headers look like 'For President Jimmy Carter, of Georgia'.
    """
    name = normalize_header(col_name_raw)
    mapping = BASE_MAP_BY_YEAR.get(year, {})

    # Try multi-word keys first (e.g., 'faith spotted eagle'), then single last names
    # Sort keys by length desc so longer phrases match first
    for key in sorted(mapping.keys(), key=len, reverse=True):
        if key in name:
            return mapping[key]

    # Also try just the last token (often the last name)
    tokens = name.split()
    if tokens:
        last = tokens[-1]
        for key in mapping.keys():
            if key == last:
                return mapping[key]

    return "OTHER"

# ===== Scraper =====
def scrape_year(year: int) -> pd.DataFrame:
    url = f"https://www.archives.gov/electoral-college/{year}"
    print(f"\nScraping {year} — {url}")
    resp = requests.get(url, timeout=30)
    resp.raise_for_status()
    soup = BeautifulSoup(resp.text, "html.parser")

    dfs = pd.read_html(StringIO(str(soup)), flavor="lxml")
    tbl = find_target_table(dfs)
    if tbl is None:
        raise RuntimeError("Could not locate Electoral College Votes by State table.")

    # Clean table & headers
    tbl.columns = flatten_cols(tbl.columns)
    first_col = tbl.columns[0]
    tbl = tbl.rename(columns={first_col: "state_raw"})
    # Drop totals rows early (by text)
    tbl = tbl[~tbl["state_raw"].astype(str).str.contains(r"\btotal\b", case=False, na=False)]
    tbl["state"] = tbl["state_raw"].astype(str).str.replace(r"\s*\*+$", "", regex=True).str.strip()

    # Identify EV total column (varies by year)
    ev_total_col = None
    for c in tbl.columns[1:]:
        if re.search(r"\belectoral\b", c, re.I) and re.search(r"\bvote|votes\b", c, re.I):
            ev_total_col = c
            break
    if ev_total_col is None:
        # fallback: pick first column that's >90% numeric
        for c in tbl.columns[1:]:
            ser = pd.to_numeric(tbl[c], errors="coerce")
            if ser.notna().mean() > 0.9:
                ev_total_col = c
                break
    if ev_total_col is None:
        raise RuntimeError("Could not identify EV total column.")

    # Coerce numerics
    for c in tbl.columns:
        if c not in ("state_raw", "state"):
            tbl[c] = pd.to_numeric(tbl[c], errors="coerce")

    numeric_cols = [c for c in tbl.columns if c not in ("state_raw","state") and pd.api.types.is_numeric_dtype(tbl[c])]
    cand_cols = [c for c in numeric_cols if c != ev_total_col]

    # Classify columns by party with normalized headers
    dem_cols, rep_cols, oth_cols = [], [], []
    for c in cand_cols:
        party = classify_candidate(c, year)
        if party == "DEM":
            dem_cols.append(c)
        elif party == "REP":
            rep_cols.append(c)
        else:
            oth_cols.append(c)

    # Debug visibility: show how columns were classified this year
    print(f"  EV total col: {ev_total_col}")
    print(f"  DEM cols: {dem_cols}")
    print(f"  REP cols: {rep_cols}")
    print(f"  OTHER cols: {oth_cols}")

    out = pd.DataFrame({
        "year": year,
        "state": tbl["state"],
        "ev_total": tbl[ev_total_col]
    })

    out["ev_dem"]   = tbl[dem_cols].sum(axis=1, numeric_only=True) if dem_cols else 0
    out["ev_rep"]   = tbl[rep_cols].sum(axis=1, numeric_only=True) if rep_cols else 0
    out["ev_other"] = tbl[oth_cols].sum(axis=1, numeric_only=True) if oth_cols else 0

    # Keep only real rows (some junk header rows have NaN/0 totals)
    out = out.dropna(subset=["ev_total"])
    out = out[out["ev_total"] > 0]

    # Balance check & correction (protect against unclassified columns)
    diff = out["ev_total"] - (out["ev_dem"] + out["ev_rep"] + out["ev_other"])
    if (diff != 0).any():
        # Add any leftover to OTHER (rare; indicates an unmapped column)
        out["ev_other"] = (out["ev_other"] + diff).clip(lower=0)

    # Integers
    out[["ev_total","ev_dem","ev_rep","ev_other"]] = out[["ev_total","ev_dem","ev_rep","ev_other"]].round(0).astype("Int64")

    # Sort & return
    out = out.sort_values(["year","state"]).reset_index(drop=True)
    return out

def main():
    years = list(range(1976, 2025, 4))
    frames = []
    for y in years:
        try:
            frames.append(scrape_year(y))
        except Exception as e:
            print(f"  ERROR {y}: {e}")
        time.sleep(0.8)

    if not frames:
        print("No data scraped.")
        return

    df = pd.concat(frames, ignore_index=True)

    # Normalize D.C. naming (rare variants)
    dc_map = {
        'district of columbia':'District of Columbia',
        'washington, dc':'District of Columbia',
        'washington, d.c.':'District of Columbia',
        'd.c.':'District of Columbia',
        'dc':'District of Columbia'
    }
    df["state"] = df["state"].str.strip()
    df["state"] = df["state"].replace(dc_map)

    # Sanity check
    bad = df.eval("ev_dem + ev_rep + ev_other != ev_total")
    if bad.any():
        print("⚠️ rows not summing to ev_total (inspect):")
        print(df.loc[bad, ["year","state","ev_dem","ev_rep","ev_other","ev_total"]])

    df.to_csv("nara_electoral_votes_by_party_1976_2024.csv", index=False)
    print("\n✅ Saved: nara_electoral_votes_by_party_1976_2024.csv")
    print(f"Rows: {len(df)} | Years: {sorted(df['year'].unique().tolist())}")

if __name__ == "__main__":
    main()


In [ ]:
def scrape_year(year: int) -> pd.DataFrame:
    import textwrap
    url = f"https://www.archives.gov/electoral-college/{year}"
    print(f"\nScraping {year} — {url}")
    resp = requests.get(url, timeout=30)
    resp.raise_for_status()
    soup = BeautifulSoup(resp.text, "html.parser")

    dfs = pd.read_html(StringIO(str(soup)), flavor="lxml")
    tbl = find_target_table(dfs)
    if tbl is None:
        raise RuntimeError("Could not locate Electoral College Votes by State table.")

    # Flatten header
    tbl.columns = flatten_cols(tbl.columns)
    first_col = tbl.columns[0]
    tbl = tbl.rename(columns={first_col: "state_raw"})
    tbl = tbl[~tbl["state_raw"].astype(str).str.contains(r"\btotal\b", case=False, na=False)]
    tbl["state"] = tbl["state_raw"].astype(str).str.replace(r"\s*\*+$", "", regex=True).str.strip()

    # Identify EV total column
    ev_total_col = None
    for c in tbl.columns[1:]:
        if re.search(r"\belectoral\b", c, re.I) and re.search(r"\bvote|votes\b", c, re.I):
            ev_total_col = c
            break
    if ev_total_col is None:
        for c in tbl.columns[1:]:
            ser = pd.to_numeric(tbl[c], errors="coerce")
            if ser.notna().mean() > 0.9:
                ev_total_col = c
                break

    for c in tbl.columns:
        if c not in ("state_raw","state"):
            tbl[c] = pd.to_numeric(tbl[c], errors="coerce")

    numeric_cols = [c for c in tbl.columns if c not in ("state_raw","state") and pd.api.types.is_numeric_dtype(tbl[c])]
    cand_cols = [c for c in numeric_cols if c != ev_total_col]

    # === NEW DEBUG SECTION ===
    print(f"\n🧭 {year} Candidate columns:")
    for c in cand_cols:
        print(f"  Raw: {repr(c)}  →  Normalized: {repr(normalize_header(c))}  →  Classified as: {classify_candidate(c, year)}")
    print("")

    # normal processing but keep printouts
    dem_cols, rep_cols, oth_cols = [], [], []
    for c in cand_cols:
        party = classify_candidate(c, year)
        if party == "DEM":
            dem_cols.append(c)
        elif party == "REP":
            rep_cols.append(c)
        else:
            oth_cols.append(c)

    print(f"  DEM cols: {dem_cols}")
    print(f"  REP cols: {rep_cols}")
    print(f"  OTHER cols: {oth_cols}")

    out = pd.DataFrame({
        "year": year,
        "state": tbl["state"],
        "ev_total": tbl[ev_total_col]
    })

    out["ev_dem"]   = tbl[dem_cols].sum(axis=1, numeric_only=True) if dem_cols else 0
    out["ev_rep"]   = tbl[rep_cols].sum(axis=1, numeric_only=True) if rep_cols else 0
    out["ev_other"] = tbl[oth_cols].sum(axis=1, numeric_only=True) if oth_cols else 0

    out = out.dropna(subset=["ev_total"])
    out = out[out["ev_total"] > 0]
    out[["ev_total","ev_dem","ev_rep","ev_other"]] = out[["ev_total","ev_dem","ev_rep","ev_other"]].round(0).astype("Int64")
    return out
scrape_year(2016)